### Generate BIDS-formatted folder structure for fmriprep

Note that we use a modified version of the T1w-scans as input to ensure skull stripping within fmriprep works.


Pipeline:
1. Create BIDS-formatted folder structure for fmriprep, ensuring that there is a single T1w-scan per subject (ses-me, otherwise ses-se)
2. Multiply T1w image by $\frac{inv2}{inv2+\gamma}$. Gamma is currently set to 100 (determined experimentally). This procedure removes the background noise while minimizing the amount of remaining bias in the image. See https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4059664/ for rationale behind the used equation.
4. Run fmriprep

In [2]:
import os
import numpy as np
import shutil
import glob
import nibabel as nib
from nilearn import plotting, masking
%matplotlib inline

In [62]:
raw_dir = './data/raw/bids'
fmriprep_raw_dir = './data/raw/bids_fmriprep'
os.makedirs(fmriprep_raw_dir, exist_ok=True)

In [63]:
fns = os.listdir(raw_dir)
fns.sort()
fns = [os.path.join(raw_dir, fn) for fn in fns]

for fn in fns:
    # files can be just copied
    if os.path.isfile(fn):
        this_file = fn.split('/')[-1]
        shutil.copy2(fn, os.path.join(fmriprep_raw_dir, this_file))
    else:
        subj_id = fn.split('/')[-1]
        print()
        print(subj_id, end=', ')
        
        subj_dir = os.path.join(fmriprep_raw_dir, subj_id)
        os.makedirs(subj_dir, exist_ok=True)
        
        for ses in ['ses-me', 'ses-se']:
            print(ses, end='... ')
            # copy func, fmap
            if os.path.isdir(os.path.join(fn, ses)):
                shutil.copytree(os.path.join(fn, ses), os.path.join(subj_dir, ses))
        
        if os.path.isdir(os.path.join(fn, 'ses-me', 'anat')):
            # ses-me has anatomy, remove ses-ses anatomy if this also exists
            if os.path.isdir(os.path.join(fn, 'ses-se', 'anat')):
                shutil.rmtree(os.path.join(subj_dir, 'ses-se', 'anat'))


sub-01, ses-me... ses-se... 
sub-02, ses-me... ses-se... 
sub-03, ses-me... ses-se... 
sub-04, ses-me... ses-se... 
sub-05, ses-me... ses-se... 
sub-06, ses-me... ses-se... 
sub-07, ses-me... ses-se... 
sub-08, ses-me... ses-se... 
sub-09, ses-me... ses-se... 
sub-10, ses-me... ses-se... 
sub-11, ses-me... ses-se... 
sub-12, ses-me... ses-se... 
sub-13, ses-me... ses-se... 
sub-14, ses-me... ses-se... 
sub-15, ses-me... ses-se... 
sub-16, ses-me... ses-se... 
sub-17, ses-me... ses-se... 
sub-18, ses-me... ses-se... 

In [64]:
def t1w_reduce_noise(t1w_path, inv2_path, gamma=100):
    ### OVERWRITES T1w images!
    t1w = nib.load(t1w_path)
    inv2 = nib.load(inv2_path)
    
    t1w_data = t1w.get_data()
    inv2_data = inv2.get_data()
    
    t1w_data = t1w_data * (inv2_data / (inv2_data+gamma))
    new_t1w = nib.Nifti1Image(t1w_data, t1w.affine)
    anat_dir = '/'.join(t1w_path.split('/')[:-1])
    os.system('touch {}/.t1w_modified'.format(anat_dir))
    nib.save(new_t1w, t1w_path)

In [65]:
# generate "new" T1w-images
fns = os.listdir(fmriprep_raw_dir)
fns.sort()
fns = [os.path.join(fmriprep_raw_dir, fn) for fn in fns]
cmds = []

for fn in fns:
    if os.path.isfile(fn):
        continue
    else:
        subj_id = fn.split('/')[-1].split('-')[-1]
        print(subj_id, end=', ')
        
        if os.path.isdir(os.path.join(fn, 'ses-me', 'anat')):
            # ses-me has anatomy
            anat_location = os.path.join(fn, 'ses-me', 'anat')
            anat_ses = 'me'
        else:
            # anatomy is from ses-se
            anat_location = os.path.join(fn, 'ses-se', 'anat')
            anat_ses = 'se'

        # generate "T1w" img
        if os.path.exists(os.path.join(anat_location, '.t1w_modified')):
            continue
        t1w_path = os.path.join(anat_location, 'sub-{}_ses-{}_T1w.nii'.format(subj_id, anat_ses))
        inv2_path = os.path.join(anat_location, 'sub-{}_ses-{}_inv-2_MPRAGE.nii'.format(subj_id, anat_ses))
        t1w_reduce_noise(t1w_path, inv2_path, gamma=100)

01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12, 13, 14, 15, 16, 17, 18, 

In [3]:
T1w_imgs = glob.glob('./data/raw/bids_fmriprep/sub-*/ses-*/anat/*_T1w.nii.gz')
T1w_imgs.sort()

for T1 in T1w_imgs:
    print(T1)
    plotting.plot_anat(T1)

Afterwards, we run fmriprep. Note that I used a slightly modified version of fmriprep 1.2.6. That is, since fmriprep 1.2, it automatically generates an optimal combination of the multiple echos in an ME dataset. I did not want this, rather, I wanted it to simply process each echo separately (except for HMC - which works better with the first echo only).

Fmriprep was run on the LISA system in Amsterdam. The docker image used is `stevenm/mefmri_1.2.6.2`

In [ ]:
# !fmriprep ./data/raw/bids_fmriprep ./data/deriv/fmriprep_126_run2 participant \
# --participant-label 01 \
# -w ./workflow_folders \
# --n_cpus 10 \
# --mem-mb 100000 \
# --anat-only \
# -v \
# --output-space T1w template \
# --template-resampling-grid native > $(date +"run_%Y%m%d_%H%M%S.txt")

/usr/local/miniconda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/mi

/usr/local/miniconda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/miniconda/lib/python3.7/site-packages/nilearn/datasets/neurovault.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it 